In [1]:
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

## Self-Attention - Encoder

Step-by-step bb - ignoring some details like standardization and dropout.

In [2]:
# toy example
torch.manual_seed(42)
    
# Let's assume we have a sequence of 4 tokens with embedding dimension 8
seq_len = 4
embed_dim = 8

# Create random embeddings to simulate a simple sentence
# In a real scenario, these would be the result of embedding words
x = torch.randn(seq_len, embed_dim)
print(f"Input tensor shape: {x.shape}")

# Sample token list for visualization
tokens = ["The", "cat", "sat", "down"]

Input tensor shape: torch.Size([4, 8])


In [3]:
Wq = nn.Linear(embed_dim, embed_dim, bias=False)  # query 
Wk = nn.Linear(embed_dim, embed_dim, bias=False)  # keys
Wv = nn.Linear(embed_dim, embed_dim, bias=False)  # values

These are the core building-blocks of the self-attention head. Each may learn *different things* during training.\
The nomenclature of "Queries" and "Keys" is irksome to me. Let's avoid reading into these meanings and just think about the mathematical relationships.

Each of these `Wq`, `Wk`, `Wv` are matrix representations of linear operators on a vector space of dimension `embed_dim`. This means that each maps the *coordinates* of the column vectors in `x` to a different set of *coordinates* in the same space.\
Through the process of training it is possible that the matrices come to represent linear projections into smaller subspaces of the full `embed_dim` dimensional vector space, but I think that in principal we begin with operators.


Here's the basic forward pass:
1. First, we take the compositions $W_q(x)$ and $W_k(x)$, effectively applying the linear operator mappings represented by both matrices on the input `x`. This is a linear transformation in the `embed_dim`-dimensional embedding space. So, each column vector (token) in `x` is taken to a different column vector within the same space.

2. Next, we multiply $W_q(x) \cdot W_k^\intercal(x)$. This is the inner-product (dot-product) of the query matrix and the dual map of the keys matrix. The result is an `seq_len x seq_len` matrix with elements corresponding to the dot-products betweent the column vectors of `Wq` and `Wk`. In other words, we get a matrix comprised of the dot-products between the $W_q(x)$ mappings in embedding space and the $W_k(x)$ mappings in embedding space. These dot-products reflect the cosine-similarity between the row vectors (token) of `Wq` and the column-vectors (tokens) `Wk`. So, large values represent greater similarities between the mappings.
    - This is conceptually significant. The mappings represented by `Wq` and the mappings represented by `Wk` are both learnable through backpropogation. This means that they may be learned *separately*. Thus, the cosine-similarity between the key-mapping for `'cat'` and the query-mapping for `'sat'` may be different from the cosine-similarity between the query-mapping for `'cat'` and the key-mapping for `'sat'`
        - This allows us to *"attend to"* tokens differently depending on whether they are query-mappings or key-mappings
        - This is significant because it allows us to learn representations of *asymmetric* relationships
        - **This asymmetry will become clear once we apply *masking*** which prevents previous tokens from attending to future tokens
    - I will refer to this `seq_len x seq_len` matrix as the attention scores matrix denoted $A$
    - I will denote the `seq_len x embed_dim` matrices of query and key coordinates as $Q \coloneqq W_q(x)$ and $K \coloneqq W_k(x)$

3. Next, we apply *masking* to the the attention scores matrix $A$ such that only cosine similarities for previous tokens are included for each token. Specifically, when we wish to ensure that only past tokens are attended to, we have an ***Encoder*** architecture. Typically, this is done by setting the elements of the upper-triangle of $A$ to $-\infty$ (which map to $0$ via softmax in step 4).\
\
 I think the most effective way to conceptualize this is by considering each column-vector (token) in $Q$ and $K$. In terms of dot-products, we can think of row $j$ of $A$ as the dot products of the $j^{\text{th}}$ token-mapping in $Q$ with each token-mapping in $K^\intercal$. So, the attention scores matrix with masking will look like: $$
 A = \begin{bmatrix} 
 Q_{1, \cdot}\cdot K_{\cdot, 1} \ \ \ \ \ \ \ \ \ \ -\infty \ \ \ \ \ \ \ \ -\infty  \ \ \ \ -\infty \\
 Q_{2, \cdot}\cdot K_{\cdot, 1} \ \ \ \ \ Q_{2, \cdot}\cdot K_{\cdot, 2} \ \ \ \ \ \ -\infty \ \ \ \ -\infty \\
 Q_{3, \cdot}\cdot K_{\cdot, 1} \ \ \ \ \ Q_{3, \cdot}\cdot K_{\cdot, 2} \ \ \ \ Q_{3, \cdot}\cdot K_{\cdot, 3} \  -\infty \\
 Q_{4, \cdot}\cdot K_{\cdot, 1} \ \ Q_{4, \cdot}\cdot K_{\cdot, 2} \ \ Q_{4, \cdot}\cdot K_{\cdot, 3} \ \ Q_{4, \cdot}\cdot K_{\cdot, 4}
 \end{bmatrix}$$ 
 Here, $K$ should really be $K^\intercal$...........\
 So, each query mapping *only* is composed with the key mappings of sequence positions *less that or equal to* its own sequence position. I.e. $$A_{j,k} = Q_{j, \cdot} \cdot K^\intercal_{\cdot, k}: \ k \le j$$
 Now, consider the asymmetric relationships. The token-mapping in column-vector $K_{\cdot, 1}$ may *attend to* (i.e. learn relationships with) each token-mapping in the row-vectors $Q_{1, \cdot}, \cdots Q_{4, \cdot}$. However, the token-mapping in row-vector $Q_{1, \cdot}$ may *only* attend to (i.e. learn relationships with) the column-vector $K_{\cdot, 1}$

4. Next, we apply a softmax transformation *over the column-vectors* of $A$ to convert the dot-products in the attention score matrix $A$ into probability weights. This applies a non-linearity and also creates a "winner-take-most" maximization. It suppresses low similarity attention scores down towards $0$ while maximizing high similarity scores up towards $1$. The strength of this maximization for entries in column $k$ of $A$ is *wholly dependent upon* the values in column $k$. So, weak similarity scores may be inflated up towards $1$ is all the other similarity scores in column $k$ are even weaker! **THIS IS IMPORTANT** This ensures that meaningful relationships will be learned between each query-key-pair even when their initial mappings in embedding space have *low semantic similarity*.

5. Finally, we multiply the softmax-maximized masked attention scores matrix by the *value* matrix $V \coloneqq W_v(x)$. This value matrix *preserves* some of the original semantic information from the embedded token representation $x$.In a sense, the value matrix $V$ represents "what information" is passed forwards while $Q$ and $K$ represent "how much of each value" should be passed forwards. Now, in the backwards pass, the weights in the matrix representation of the value-map $W_v$ are updated in part by the attention in scores in $A$: $$W_v = W_v - \text{lr} \cdot \frac{\partial L}{\partial W_v} = W_v - \text{lr} \cdot \bigg(\frac{\partial L}{\partial \text{Output}} \cdot A\bigg) \cdot x^\intercal$$
This means that the *training signal* for $V$ is directly shaped by the attention patterns in $A$ despite $V$ itself having no direct influence on the attention patterns.\
In essence, the value matrix $V$ *learns* to create representations that, *when weighted according to the attention patterns* in $A$, produce effective outputs. If certains tokens receive higher attention weights, then the corresponding token-mappings in $W_v$ will be updated more aggressively. **BOOM**\
There is a feedback loop here:
    - If a token's value representation is useful, models learn to attend to it more
    - If attention to a token is consistently high, its value representation becomes more refined

In [4]:
x = x  # (seq_len, embed_dim) -- assumes input x is already embedded

Q = Wq(x)  # (seq_len, embed_dim)
K = Wk(x)  # (seq_len, embed_dim)

attn_scores = Q @ K.T  # (seq_len, seq_len)

In [ ]:
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
mask

tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

In [7]:
attn_scores.masked_fill(mask, float("-inf"))

tensor([[ 1.2523,    -inf,    -inf,    -inf],
        [-0.6610,  0.1095,    -inf,    -inf],
        [ 0.3460, -0.5986, -0.5688,    -inf],
        [-0.2937,  0.9701,  1.8560,  0.0310]], grad_fn=<MaskedFillBackward0>)

There's one last important element which we've neglected for the sake of simplicity - standardization. Before applying the softmax transformation over the column vectors of the attention scores matrix $A$. This prevents very large values from completely dominating the softmax transformation. Such large values may be arbitrary when some row-vecotrs in $Q$ and column vectors in $K^\intercal$ happen to have a large scale relative to the other row and column vectors. So, we standardize scales to prevent the arbitrary saturation of the softmax transform.

### Class Wrapper - Single-Head

In [11]:
from dataclasses import dataclass

@dataclass
class Encoder(nn.Module):
    d_emb: int
    seq_len: int

    def __post_init__(self):
        super().__init__()
        self.Wq = nn.Linear(self.d_emb, self.d_emb, bias=False)  # query weights
        self.Wk = nn.Linear(self.d_emb, self.d_emb, bias=False)  # key weights
        self.Wv = nn.Linear(self.d_emb, self.d_emb, bias=False)  # value weights

    def forward(self, x):
        Q = self.Wq(x)  # (seq_len, d_emb) -- map x into the query coordinate-space
        K = self.Wk(x)  # (seq_len, d_emb) -- map x into the key coordinate-space
        V = self.Wv(x)  # (seq_len, d_emb) -- map x into the value coordinate-space

        # get attention scores
        A = Q @ K.T  # (seq_len, seq_len)

        # mask out the upper triangular part of the attention scores
        mask = torch.triu(torch.ones(self.seq_len, self.seq_len), diagonal=1).bool()
        A.masked_fill(mask, float("-inf"))

        # standardize the attention scores
        A = (A - A.mean(dim=-1, keepdim=True)) / (A.std(dim=-1, keepdim=True) + 1e-8)

        # apply softmax to get the attention probabilities
        A = F.softmax(A, dim=-1)

        # apply the attention probabilities to the values
        return A @ V  # (seq_len, d_emb)

## Single-Head vs. Multi-Head

The attention mechanism described so far is called a *Single-Head* encoder. This architecture has one set of query, key, and value matrices. And augmented architecture is *Multi-Head* attention, in which we split the embedding space into subspaces of dimensions `d_emb / n_heads`. This has several benefits:
1. We may run more passes at once by parrallelizing
2. We may process passes more quickly by working in lower-dimensional spaces
3. Each head has *it's own* query, key, and value matrices, allowing each to learn *different* attention patterns

In the multi-head architecture we specify the overall dimension of the input embedding space as `d_model`. This is because the spaces of each attention head are lower dimensional subspaces that we *project into*. Each has dimension `d_head = d_model / n_heads`. 

To output from the multi-head attention stack, we *concatenate* the output from each attention head and apply a linear output transformation. The ouptut of each attention head is a `seq_len x d_head`. The concatenation is functionally the same as taking a *direct sum* of *row-vectors* of the outputs, such that each row vector has a finaly dimension of `d_head * n_heads`, and the final output matrix has dimension `seq_len x d_head * n_heads = seq_len x d_model`.

In [16]:
e1 = Encoder(d_emb=6, seq_len=4)
e2 = Encoder(d_emb=6, seq_len=4)

x = torch.randn(4, 6)
a1 = e1(x)
a2 = e2(x)

print(a1, a2)

c = torch.cat([a1, a2], dim=-1)
print(c, c.shape)

tensor([[-0.2663, -0.1443,  0.4039,  0.5612,  0.0865, -0.4306],
        [-0.0622,  0.3100,  0.1371,  0.2533,  0.0344, -0.2972],
        [ 0.2056,  0.3186,  0.0188, -0.3248,  0.0744, -0.1199],
        [-0.2097, -0.3447,  0.4588,  0.3800,  0.1312, -0.3922]],
       grad_fn=<MmBackward0>) tensor([[ 0.4995, -0.4797,  0.3424,  0.2441,  0.1585, -0.4040],
        [ 0.4538, -0.1359,  0.5625, -0.0490, -0.0443, -0.0250],
        [ 0.0143,  0.2704, -0.2241, -0.3108, -0.0804,  0.1552],
        [-0.0107,  0.2689, -0.2915, -0.3011, -0.0642,  0.1340]],
       grad_fn=<MmBackward0>)
tensor([[-0.2663, -0.1443,  0.4039,  0.5612,  0.0865, -0.4306,  0.4995, -0.4797,
          0.3424,  0.2441,  0.1585, -0.4040],
        [-0.0622,  0.3100,  0.1371,  0.2533,  0.0344, -0.2972,  0.4538, -0.1359,
          0.5625, -0.0490, -0.0443, -0.0250],
        [ 0.2056,  0.3186,  0.0188, -0.3248,  0.0744, -0.1199,  0.0143,  0.2704,
         -0.2241, -0.3108, -0.0804,  0.1552],
        [-0.2097, -0.3447,  0.4588,  0.3800, 

We then apply another linear transformation upon this output, mapping the output row vectors onto the same `d_model`-dimensional space. This transformation is an ***Endomorphism*** (a mapping onto the same space, aka a linear operator, that may not be bijective). Since this transformation has learnable parameters, it allows us to learn which representations from the multiple attention heads are most useful in a more efficient way that simply passing the concatenated output forward.

### Class Wrapper - Multi-Head

In [17]:
@dataclass
class MultiHeadEncoder(nn.Module):
    d_model: int
    seq_len: int
    n_heads: int

    def __post_init__(self):
        super().__init__()
        self.heads = nn.ModuleList([
            Encoder(self.d_model // self.n_heads, self.seq_len) 
            for _ in range(self.n_heads)
        ])

        self.Wo = nn.Linear(self.d_model, self.d_model)

    def forward(self, x):
        out_cat = torch.cat([head(x) for head in self.heads], dim=-1)
        return self.Wo(out_cat)